## Рассуждения

### Идея 1: взять токенайзер и побить каждый из запросов на токены.

Эта идея сразу отпала, потому что большая часть токенайзеров предполагает, что слова в корпусе будут разделены пробелом.

В добавок, ну, вот побили мы на токены, а как объединять эти токены в слова? (чтобы понять где должны стоять пробелы)

### Идея 2: Накопление Буфера

Давайте отталкиваться от недостатков первой идеи, а именно как объединять эти токены в слова?

Сделаем буфер, куда будем добавлять каждый символ из запроса. Как только мы соберем несколько кандидатов на слова, выберем из них наиболее длинное и поставим после него пробел.

Эта идея подразумевает наличие корпуса, в который мы будем подглядывать, чтобы понять является набор символов словом или нет.

Поэтому я смотался на kaggle и взял OpenCorpora-russan https://www.kaggle.com/datasets/rtatman/opencorpora-russian

Реализацию 2-й идеи можно найти в разделе артефакты **cumulative_imputer_max**.

P.S. Почему выбирается максимальное по длине слово?

Это эвристика, основанная на том, что мы хотим полностью разбить запрос на слова. Т.е. нам в каком-то смысле выгодно откусыввать большие куски запроса.



### Докручиваем идею 2

**Успешные** примеры

отдамдаромкошку -> отдам даром кошку

новыйдивандоставканедорого -> новый диван доставка недорого

**Неудачные** примеры

работавмосквеудаленно -> работав москве удал е н но

сдаюквартирусмебельюитехникой -> сдаю квартиру см е белью ит е х никой

Проблема с неудачными примерами очевидна. Алгоритм работает по эвристике, которая никогда не выберет **работа**, если можно выбрать деепричастие **работав** или **с**, если можно выбрать **см**.

Если мы хотим выбирать комбинации слов, которые наиболее похожи на то, что ввел человек, то нам не избежать использование частот/статистики употребеления того или иного слова.

Вряд ли удастся найти такой список, который полность совпал бы с нашим словарем, поэтому давайте возьмем маленькую языковую модель и будем считать перплексию, вместо максимальной длины слова.

У странных слов по типу **в москве удал е** пералексия будет больше, чем у **в москве удаленно**

Модельку взял с hf https://huggingface.co/kplro/rugpt3small_based_on_gpt2-l2_russian_10_epoch какой-то небольшой чекпоинт, который как раз подходит под наши цели

P.S. Да в идеале обучить что-то свое и поменьше на корпусе объявлений, но как proof-of-concept нам подойдет и эта.

### В разделе код представлена реализации докрученной второй идеи (Конкретно в классе Perplexity imputer)
Большая часть кода - чисто техническая перегонка из одного формата в другой и попытка в обработку спец. символов (пунктуация, цифры, знаки препинания и т.д.)

## Код

In [ ]:
!pip3 install transformers
!pip3 install pandas
!pip3 install torch torchvision

In [ ]:
!unzip archive.zip

In [1]:
from typing import List, Tuple

import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
DATA_PATH = "dataset_1937770_3.txt"
VOCAB_PATH = "dictionary.txt"
device = "cpu"

In [3]:
def data_loader(data_path, header=True):
    data = []
    col_names = []
    with open(data_path, 'r') as file:
        i = 0
        for line in file:
            if header:
                col_names = line.strip().split(",")
                header = False
                continue
            data.append([i, ",".join(line.split(",")[1:]).strip()])
            i += 1
    return pd.DataFrame(data, columns=col_names)

def vocab_loader(vocab_path):
    vocab = set()
    with open(vocab_path, 'r') as file:
        for line in file:
            if line.strip().isnumeric():
                continue
            vocab.add(line.split("\t")[0].strip().lower())
    vocab.add("бу")
    return vocab


In [4]:
vocab = vocab_loader(VOCAB_PATH)
data_df = data_loader(DATA_PATH)

In [6]:
tokenizer = AutoTokenizer.from_pretrained("kplro/rugpt3small_based_on_gpt2-l2_russian_10_epoch")
model = AutoModelForCausalLM.from_pretrained("kplro/rugpt3small_based_on_gpt2-l2_russian_10_epoch")

In [64]:
class PerplexityImputer():
    
    def __init__(self, vocab, model, tokenizer):
        # parameters
        
        # storage
        self.vocab = vocab
        self.model = model
        self.tokenizer = tokenizer
        self.letters = set("abcdefghijklmnopqrstuvwxyz-–’")
        self.digits = set("0123456789")
        self.punkt = set(".,;!?:…")
        return
    
    def get_next_variant(self, query: str, start: int = 0, amount: int = 1) -> List[List[Tuple[str, int]]]:
        """
        Функция, которая выдает amount возможных слов начинающихся с позиции start

        Args:
            query  (str) : запрос.
            start  (int) : позиция, с которой надо начинать искать слова.
            amount (int) : максимальное количество слов, которое нужно найти.

        Returns:
            List[List[Tuple[str, int]]]: Список списков кортежей в формате (слово, позиция идущая после слова).
        """

        j = start + 1
        variants = []
        while j < len(query) + 1:   # ищем в цикле следующее слово, пока не закончится запрос
            variant = query[start:j]
            if variant in self.vocab:
                variants.append([(variant, j)])
            if len(variants) == amount: # Если нашли требуемое количство слов, выходим из цикла заранее
                break
            j += 1
        
        if start < len(query) and len(variants) == 0:
            variants.append([("DEL", len(query))])
            return variants
            
        return variants
    
    def extend_variants(self, query: str, variants: List[Tuple[str, int]]) -> List[List[Tuple[str, int]]]:
        """
        Функция, которая расширяет возможные комбинации слов

        Args:
            query    (str)                   : запрос.
            variants (List[List[Tuple[str, int]]]) : Список списков кортежей в формате (слово, позиция идущая после слова).

        Returns:
            List[List[Tuple[str, int]]]: Расширенный список списков кортежей в формате (слово, позиция идущая после слова).
        
        Example:
            [[(я, 1)]] -> [[('с', 1), ('к', 2)], [('с', 1), ('ка', 3)], [('с', 1), ('кажи', 5)]]
        """

        new_variants = []
        for idx, variant in enumerate(variants): # добавляем в варианты возможные продолжения
            for next_variant in self.get_next_variant(query, start=variant[-1][1], amount=10):
                new_variants.append(variants[idx] + next_variant)
        
        return new_variants
    
    def get_texts(self, variants: List[List[Tuple[str, int]]]) -> List[Tuple[str, int]]:
        """
        Функция, которая преобразует список вариантов в список текстов

        Args:
            variants (List[Tuple[str, int]]) : Список списков кортежей в формате (слово, позиция идущая после слова).

        Returns:
            List[Tuple[str, int]]: Список кортежей в формате (текст, индекс).
        
        """
        texts = []
        for idx, variant in enumerate(variants): # распаковываем варианты
            text = ""
            for pair in variant:
                text += pair[0] + " "
            texts.append((text.rstrip(), idx))
        return texts
    
    def get_perplexity(self, text: str) -> float:
        """
        Функция, которая считает перплексию text

        Args:
            text  (str) : текст.
        Returns:
            float: Значение перплексии.
        """
        with torch.no_grad():
            encodings = self.tokenizer(text, return_tensors="pt")
            outputs = self.model(**encodings, labels=encodings.input_ids)
            loss = outputs.loss
            ppl = torch.exp(loss).item()
        return ppl

    def prune(self, variants: List[List[Tuple[str, int]]], beam_width: int) -> List[List[Tuple[str, int]]]:
        """
        Функция, которая оставляет beam_width вариантов с самой низкой перплексией

        Args:
            variants (List[List[Tuple[str, int]]]) : Список списков кортежей в формате (слово, позиция идущая после слова).

        Returns:
            List[List[Tuple[str, int]]]: Список списков кортежей в формате (слово, позиция идущая после слова).
        
        """
        pruned_variants = []
        texts = self.get_texts(variants)
        indices = [x[1] for x in sorted(texts, key=lambda x: self.get_perplexity(x[0]) / (5 * x[1] + 1) )[:beam_width]]
        for i in indices:
            pruned_variants.append(variants[i])
        return pruned_variants
    
    def sieve(self, variants):
        """
        Функция, которая отсеивает тупиковые варианты

        Args:
            variants (List[List[Tuple[str, int]]]) : Список списков кортежей в формате (слово, позиция идущая после слова).

        Returns:
            List[List[Tuple[str, int]]]: Просееные варианты.
        
        """

        sieved = []
        for variant in variants: # нашли метку "DEL" -> удаляем вариант
            if variant[-1][0] != "DEL":
                sieved.append(variant)
        return sieved
    
    def decode(self, result: List[List[Tuple[str, int]]]) -> Tuple[List[int], str]:
        """
        Функция, которая преобразует вариант в список индексов, где должны стоять пробулы и полученный текст

        Args:
            result (List[List[Tuple[str, int]]]) : Список списков кортежей в формате (слово, позиция идущая после слова).

        Returns:
            Tuple[
                List[int] : список индексов, где должны стоять пробелы
                str       : запрос с расставленными пробелами
            ]
        
        """
        
        indices = [x[1] for x in result]
        text = [x[0] for x in result]
        return indices, " ".join(text).rstrip()
    
    def query_preprocessor(self, query : str) -> Tuple[List[str], List[Tuple[int, int]], List[str]]:
        """
        Функция, ответственная за предобработку запроса.
        Определяется наличие спец. символов их положение и тип.
        Если необходимо - запрос разбивается на подзапросы

        Args:
            query (str): запрос.

        Returns:
        Tuple[
            List[str]            : подзапросы
            List[Tuple[int, int] : расположение спец. символов
            List[str]]           : типы групп спец. символов
        ]
        
        """
        

        query = query.lower()
        idxs = []
        types = []
        
        for i in range(len(query)): # классифицируем специальные символы по группам
            if query[i] in self.digits:
                types.append('d')
                idxs.append(i)
            if query[i] in self.punkt:
                types.append('p')
                idxs.append(i)
            if query[i] in self.letters:
                types.append('l')
                idxs.append(i)


        if idxs:
            compressed_types = [types[0]] 
            start = end = idxs[0]
            indices = []
            for i in range(1, len(idxs)): # оюъединяем последовательные отрезки
                if idxs[i] == idxs[i-1] + 1 and types[i] == types[i-1]:
                    end = idxs[i]
                else:
                    indices += [(start, end)] if start != end else [(start, start)]
                    start = end = idxs[i]
            
            indices += [(start, end)] if start != end else [(start, start)]

            for i in range(1, len(types)): # объединяем типы в группы
                if types[i] != compressed_types[-1]:
                    compressed_types.append(types[i])

            qs = []
            first = query[:indices[0][0]]
            if first != "":
                qs.append(first)
            for i in range(1, len(indices)): # разбиваем запросы на подзапросы
                sub_query = query[indices[i-1][1] + 1:indices[i][0]]
                if sub_query != "":
                    qs.append(sub_query)
            last = query[indices[-1][1] + 1:]
            if last != "":
                qs.append(last)
            return qs, indices, compressed_types
        
        return [query], [], []

    def query_processor(self, query: str, beam_width: int=5, is_subquery=False) -> List[List[Tuple[str, int]]]:

        """
        Функция поиска слов с помощью Beam-Search

        Args:
            query      (str)  : запрос.
            beam_width (int)  : количество кандидатов, которые остануться
            is_subquery (bool): флаг, обозначающий, является ли запрос подзапросом

        Returns:
            List[List[Tuple[str, int]]]: Список списков кортежей в формате (слово, позиция идущая после слова)
        
        """

        
        if is_subquery == True and query in self.vocab: # Если прилетает подзапрос, пытаемся закрыть его подглядыванием в словарь
            return [[(query, len(query))]]
        
        variants = self.get_next_variant(query, start=0, amount=5) # получение начальных вариантов
        saved = []
        i = 0

        while i < 20 and saved == []: # аля Beam-Search для поиска вариантов с наименьшей перплексией

            variants = self.extend_variants(query, variants)

            variants = self.sieve(variants)
            for variant in variants:
                if variant[-1][1] == len(query): # Если мы дошли до конца запроса, перед нами ответ
                    saved.append(variant)
                    self.result = saved[0]
                    return saved
            variants = self.prune(variants, beam_width)
            
            i += 1
        self.result = saved[0]
        return saved
    
    def merge_subresults(self, subresults: List[List[List[Tuple[str, int]]]], indices: List[Tuple[int, int]] , types: List[str], query: str) -> List[int]:
        """
        Функция, объединяющая ответы на подзапросы в единый ответ

        Args:
            subresults (List[List[List[Tuple[str, int]]]]) : Список ответов на подзапросы
            indices (List[Tuple[int, int]])                : расположение спец. символов
            types (List[str])                              : типы групп спец. символов


        Returns:
            List[int] - Список индексов, на месте которых должен стоять пробел
        
        """

        i, j = 0, 0 
        global_ptr = 0
        rel_idxs = []
        result = []

        # получаем относительные позиции пробелов для каждого подзапроса
        for subresult in subresults:
            idxs, _ = self.decode(subresult[0])
            rel_idxs.append(idxs)
        
        # обработка спец. символов и подзапросов
        while i < len(indices) and j < len(rel_idxs):

            rel_ptr = rel_idxs[j][0] if rel_idxs[j] else 0
            
            if indices[i][0] < global_ptr + rel_ptr: # перед нами спец. символ
                
                if indices[i][0] != 0 and (not result or result[-1] != indices[i][0]):
                    result.append(indices[i][0])  # правый пробел
                if indices[i][1] != len(query) and (not result or result[-1] != indices[i][1] + 1):
                    result.append(indices[i][1] + 1)  # левый пробел
                
                global_ptr = indices[i][1] + 1 
                i += 1
            else:
                # перед нами подзапрос
                for rel_idx in rel_idxs[j]:
                    abs_pos = rel_idx + global_ptr
                    if not result or result[-1] != abs_pos:
                        result.append(abs_pos)
                
                if rel_idxs[j]:
                    global_ptr += rel_idxs[j][-1]
                j += 1

        # обработка оставшихся спец. символов
        
        while i < len(indices):
            if indices[i][0] != 0 and (not result or result[-1] != indices[i][0]) and types[i] != 'p':
                result.append(indices[i][0])
            if indices[i][1] != len(query) and (not result or result[-1] != indices[i][1] + 1):
                result.append(indices[i][1] + 1)
            
            global_ptr = indices[i][1] + 1
            i += 1

        # обработка оставшихся подзапросов
        while j < len(rel_idxs):
            for rel_idx in rel_idxs[j]:
                abs_pos = rel_idx + global_ptr
                if not result or result[-1] != abs_pos:
                    result.append(abs_pos)
            
            if rel_idxs[j]:
                global_ptr += rel_idxs[j][-1]
            j += 1

        # удаляем последний пробел, если он выходит за границы запроса
        if result and result[-1] == len(query):
            result.pop()
        
        # знаки пунктуации съедают пробел слева от себя
        for i in range(len(query)):
            if query[i] in self.punkt:
                result.remove(i)

        return result

    def impute_whitespaces(self, query: str) -> List[int]:
        """
        Основная функция, которая просто запускает весь пайплайн.

        Args:
            query (str): запрос.

        Returns:
            List[int] : список индексов, где должны стоять пробелы
        
        """
        try:
            # пробуем обработать запрос
            query = query.lower()
            subresults = []
            subqueries, indices, types = self.query_preprocessor(query) # предобработка запроса
            for subquery in subqueries:
                subresults.append(self.query_processor(subquery, is_subquery=len(subqueries)>1 or indices != [])) # обработка запроса
            
            
            if indices == []: # если запросов не было, то просто форматируем к ответу
                return self.decode(subresults[0][0])[0]
            
            result = self.merge_subresults(subresults, indices, types, query) # если были подзапросы - объединяем и форматируем
            return result
        except: # если не получилось обработать запрос, выдаем пустой список и выводим запрос, вызваввший ошибку 
            print(query)
            return []



In [65]:
pi = PerplexityImputer(
    vocab,
    model,
    tokenizer
)

In [73]:
def create_submission(data_df, imputer, path: str = "submission.csv") -> None:
    data_df["predicted_positions"] = data_df.apply(lambda row: imputer.impute_whitespaces(row["text_no_spaces"]), axis=1)
    submission = data_df.drop(columns=["text_no_spaces"])
    submission["predicted_positions"] = submission["predicted_positions"].astype(str)
    submission.to_csv(path)

In [ ]:
create_submission(data_df, pi, "submission.csv")

якалендарьперевернул
твоядочьимойсын—
аркадийукупник—кусающийлжец
мыпьемчайвстарыхквартирах,
однасемья,двесемьи,трисемьи...
рядомсметро,центр...
всеговорят,чтомыв-месте...
стой!опаснаязона!работамозга!..
...годамидолгими.
...ночамитемными.
искалатебяночами,чами,чами,чами...
абезмузыкинамирусмертьнекрасна,
счастьеданоповстречатьильбедуеще
приказаверитьвчудеса
колотьустала.
волкиуходят...
билетводинконец—
ялюблюслушатьпесни,икостранюхатьдым
сердцебьетсядругимивершинами
сигареткаосталасьвсегоодна,нуладнона...
замыкание+бах!ивсталитрамваи
онаскажет:'а,вообще,володька,хреново!'
пришёлкколдунье:'ану-канаколдуймне!'
'легко,мойхороший,толькохлопнувладоши,
ималенькаяжизньвнутринеёоборвётся.'
привидений,говорят:'небыватьпреступленью'
сдавайся,ведьма–«ночнойдозор».
потёмнымулицамлетит«ночнойдозор».
носила«иного»вантоновомвзоре,
азначит,онбудетработатьв«дозоре».
ещебы,абыкогонеберутвзамминистры.
нозлойзавулониз«дозорадневного»
сделализсынаантона«иного».
иновоеутро,иновое'неттебя...'
'друзьяхотятпо

Алгоритм отказался размечать 37 сэмплов из 1004, о причинах подумаем позднее...

Засылаем сабмит, получем скор: Your Mean F1 = **79.266%** 🎉🎉🎉

## Артефакты

In [ ]:
def cumulative_imputer_max(query : str, vocab, vc=10, bl=10) -> list[int]:
    # Ф-ция успешно обрабатывает только часть простых запросов и является просто proof-of-concept!
    try:
        print(query)
        variants = []
        whitespaces = []
        buffer = ""
        result = []
        parsed_index = 0
        
        query = query.lower()
        i = 0
        while i < len(query):
            buffer += query[i]
            
            if buffer in vocab:
                print(buffer)
                variants.append(buffer)

            if len(variants) == vc or len(buffer) == bl or i + 1 == len(query):
                print(f"\n\nbuffer overflow : {len(buffer) == bl}\nvariant overflow : {len(variants) == vc}\n\n")
                print(variants)
                result.append(max(list(zip(map(len, variants), variants)), key=lambda x: x[0])[1])
                parsed_index += len(result[-1])
                whitespaces.append(parsed_index)
                buffer = ""
                variants = []
                i = parsed_index
                print(f"{result[-1]} : saved")
                continue


            i += 1
    except:
        return " "